In [44]:
#CONFERENCIA NECESSARIA PARA VERIFICAR SE AS NOTAS GERADAS NO MES NO sistema BATE COM AS 
#NOTAS GERADAS NO SITE DA UNIDADE EM QUESTÃO  - sistema(EXCEL) SITE(XML)
import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET
import os
from datetime import datetime
#pd.set_option('max_columns', None)
#pd.set_option("max_rows", None)

mes = 1
ano = 2022

In [45]:
#lendo a base de dados de notas faturadas no sistema para conferencia, filtros utilizados para filtrar a empresa correta,
# ano atual, o mes atual, e resetando o index da tabela para manipular a tabela mais a frente de forma mais facilitada.
sistema =pd.read_excel('C:/Users/Gregory Toledo/conferencia notas fiscais/RELATORIO SISTEMA/planilha do sistema.xlsx')
sistema = sistema[sistema['UNIDADE FATURAMENTO'] ==('CONSULTORIA DE LOGISTICAS LTDA' )]
sistema = sistema[sistema['ANO'] == (ano)]
sistema = sistema[sistema['MES'] == (mes)]
sistema = sistema.reset_index(drop=True)

In [46]:
#Acessando o arquivo XML do mes passado retirado do site de nova lima contendo todas as notas fiscais
caminho = 'C:/Users/Gregory Toledo/conferencia notas fiscais/XML'
filename = os.listdir(caminho)[0]
fullname = os.path.join(caminho, filename)
tree = ET.parse(fullname)

In [47]:
#Criando as listas para receber as informações  do XML
nfe = []
nf = []
data_emissao = []
serv = []
valor_bruto = []
valor_liquido = []
PIS = []
COFINS = []
CSLL = []
IR = []
ISS = []
nome_cliente = []
razao = []

#Coletando as informações e adicionando as listas
for elm in tree.findall(".//Numero"):
    nfe.append(elm.text)
for elm in tree.findall(".//Competencia"):
    data_emissao.append(elm.text)    
for elm in tree.findall(".//ItemListaServico"):
    serv.append(elm.text)   
for elm in tree.findall(".//ValorServicos"):
    valor_bruto.append(elm.text)   
for elm in tree.findall(".//ValorLiquidoNfse"):
    valor_liquido.append(elm.text)   
for elm in tree.findall(".//ValorPis"):
    PIS.append(elm.text) 
for elm in tree.findall(".//ValorCofins"):
    COFINS.append(elm.text) 
for elm in tree.findall(".//ValorCsll"):
    CSLL.append(elm.text) 
for elm in tree.findall(".//ValorIr"):
    IR.append(elm.text) 
for elm in tree.findall(".//ValorIssRetido"):
    ISS.append(elm.text) 
for elm in tree.findall(".//RazaoSocial"):
    nome_cliente.append(elm.text)

# a tag<numero> é utilizada para numero da nota e numero de endereços,
#para que seja colhido somente o numero das notas fiscais é preciso que usemos o while abaixo
#que ignora os numeros de endereços contidos na nota, pulando de 4 em 4 indices e agregando a uma
#segunda lista filtrada com somente numeros de NF's
cont = 0
while (cont < len(nfe)):
    nf.append(nfe[cont])
    cont = cont + 4

#a tag<RazaoSocial> é utilizada para os clientes e para o emissor da nf(no caso a inter), utilizamos
# o while abaixo ignora a razão social da inter e colhe a do cliente para a lista filtrada com somente
# clientes

cont = 1
while (cont < len(nome_cliente)):
    razao.append(nome_cliente[cont])
    cont = cont + 2

In [48]:
#Dataframes utilizados para organizarmos todas as informações
df = pd.DataFrame(columns=['NF','DATA EMISSÃO','COMPETENCIA','ITEM SERVIÇO','VALOR BRUTO SISTEMA', 'ISS SISTEMA','PIS SISTEMA','COFINS SISTEMA', 'CSLL SISTEMA','IR SISTEMA','VALOR LIQUIDO SISTEMA','VALOR BRUTO SITE', 'ISS SITE','PIS SITE', 'COFINS SITE','CSLL SITE','IR SITE','VALOR LIQUIDO SITE','DIFERENÇA','CLIENTE SISTEMA','CLIENTE SITE','COMPARAÇÃO NOME'])
df_site = pd.DataFrame(columns=['NF', 'COMPETENCIA', 'ITEM SERVIÇO','VALOR BRUTO SITE','ISS SITE','PIS SITE','COFINS SITE','CSLL SITE','IR SITE','VALOR LIQUIDO SITE','CLIENTE SITE'])

In [49]:
#Etapa muito importante para que juntemos todas as notas geradas no site e no sistema, e removermos as duplicatas
#podem ter notas geradas somente no sistema ou no site, e a conferencia serve para identificar as mesmas e tratar esses erros

#lista com as notas geradas e validas do sistema
nf_sistema = sistema['NR.NOTA DE SERVIÇO'].tolist()
#convertendo as notas do site de string para inteiro
nf = list(map(int,nf))
#juntando todas as notas em uma lista só
nf_def = nf_sistema + nf
#removendo as duplicatas presentes na nova lista e reformando em lista
nf_def = list(set(nf_def))
#inclundo todas as notas no dataframe principal
df['NF'] = nf_def

In [50]:
#df_ste é um dataframe secundario criado para auxiliar o df a colher informações referentes ao site de maneira mais 
#adequada e organizada, agregando todas as listas com a informação do site no dataframe.

df_site['NF'] = nf
df_site['COMPETENCIA'] = data_emissao
df_site['ITEM SERVIÇO'] = serv
df_site['VALOR BRUTO SITE'] = list(map(float,valor_bruto))
df_site['VALOR LIQUIDO SITE'] = list(map(float,valor_liquido))
df_site['ISS SITE'] = list(map(float,ISS))
df_site['PIS SITE'] = list(map(float,PIS))
df_site['COFINS SITE'] = list(map(float,COFINS))
df_site['CSLL SITE'] = list(map(float,CSLL))
df_site['IR SITE'] = list(map(float,IR))
df_site['CLIENTE SITE'] = razao

In [51]:
#inserindo as informações do sistema(excel) no df de acordo com as notas fiscais
for cont4 in range(len(df)):
    for cont5 in range(len(sistema)):
        if df['NF'].loc[cont4] == sistema['NR.NOTA DE SERVIÇO'].loc[cont5] :
            df['DATA EMISSÃO'].loc[cont4] = sistema['DT. EMISSAO'].loc[cont5]
            df['VALOR BRUTO SISTEMA'].loc[cont4] = sistema['VLR.RECEITA'].loc[cont5]
            df['ISS SISTEMA'].loc[cont4] = sistema['VLR.ISS'].loc[cont5]
            df['PIS SISTEMA'].loc[cont4] = sistema['VLR.PIS'].loc[cont5]
            df['COFINS SISTEMA'].loc[cont4] = sistema['VLR.COFINS'].loc[cont5]
            df['CSLL SISTEMA'].loc[cont4] = sistema['VLR.CSLL'].loc[cont5]
            df['IR SISTEMA'].loc[cont4] = sistema['VLR.IRRF'].loc[cont5]
            df['VALOR LIQUIDO SISTEMA'].loc[cont4] = sistema['VLR LIQUIDO'].loc[cont5]
            df['CLIENTE SISTEMA'].loc[cont4] = sistema['CLIENTE'].loc[cont5]

C:\Users\Gregory Toledo\AppData\Local\Temp\ipykernel_420\717441054.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['DATA EMISSÃO'].loc[cont4] = sistema['DT. EMISSAO'].loc[cont5]
C:\Users\Gregory Toledo\AppData\Local\Temp\ipykernel_420\717441054.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['VALOR BRUTO SISTEMA'].loc[cont4] = sistema['VLR.RECEITA'].loc[cont5]
C:\Users\Gregory Toledo\AppData\Local\Temp\ipykernel_420\717441054.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/u

In [52]:
#inserindo as informações do df_site no df de acordo com as notas fiscais
cont4 = 0
cont5 = 0
for cont4 in range(len(df)):
    for cont5 in range(len(df_site)):
        if df['NF'].loc[cont4] == df_site['NF'].loc[cont5] :
            df['COMPETENCIA'].loc[cont4] = df_site['COMPETENCIA'].loc[cont5]
            df['ITEM SERVIÇO'].loc[cont4] = df_site['ITEM SERVIÇO'].loc[cont5]
            df['VALOR BRUTO SITE'].loc[cont4] = df_site['VALOR BRUTO SITE'].loc[cont5]
            df['ISS SITE'].loc[cont4] = df_site['ISS SITE'].loc[cont5]
            df['PIS SITE'].loc[cont4] = df_site['PIS SITE'].loc[cont5]
            df['COFINS SITE'].loc[cont4] = df_site['COFINS SITE'].loc[cont5]
            df['CSLL SITE'].loc[cont4] = df_site['CSLL SITE'].loc[cont5]
            df['IR SITE'].loc[cont4] = df_site['IR SITE'].loc[cont5]
            df['VALOR LIQUIDO SITE'].loc[cont4] = df_site['VALOR LIQUIDO SITE'].loc[cont5]
            df['CLIENTE SITE'].loc[cont4] = df_site['CLIENTE SITE'].loc[cont5]

C:\Users\Gregory Toledo\AppData\Local\Temp\ipykernel_420\3161182150.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['COMPETENCIA'].loc[cont4] = df_site['COMPETENCIA'].loc[cont5]
C:\Users\Gregory Toledo\AppData\Local\Temp\ipykernel_420\3161182150.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ITEM SERVIÇO'].loc[cont4] = df_site['ITEM SERVIÇO'].loc[cont5]
C:\Users\Gregory Toledo\AppData\Local\Temp\ipykernel_420\3161182150.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_

In [53]:
#substituindo os NaN da coluna em valor numerico '0.00'  
df['ISS SISTEMA'] = df['ISS SISTEMA'].replace(np.nan, 0.00, regex=True)
df['PIS SISTEMA'] = df['PIS SISTEMA'].replace(np.nan, 0.00, regex=True)
df['COFINS SISTEMA'] = df['COFINS SISTEMA'].replace(np.nan, 0.00, regex=True)
df['CSLL SISTEMA'] = df['CSLL SISTEMA'].replace(np.nan, 0.00, regex=True)
df['IR SISTEMA'] = df['IR SISTEMA'].replace(np.nan, 0.00, regex=True)
#Calculando a diferença entre as notas no valor liquido:
#for cont8 in range(df):
    #df['DIFERENÇA'].loc[cont8] = float(df['VALOR LIQUIDO SISTEMA'].loc[cont8]) - float(df['VALOR LIQUIDO SITE'].loc[cont8])
 #   if df['CLIENTE SISTEMA'].loc[cont8] == df['CLIENTE SITE'].loc[cont8]:
  #      df['COMPARAÇÃO NOME'] = 'ok'
   # else:
    #    df['COMPARAÇÃO NOME'] = 'nome divergente'
    

#ordenando por numero da NF
df = df.sort_values(by='NF')

In [54]:
df['VALOR LIQUIDO SISTEMA'] = df['VALOR BRUTO SISTEMA'] - df['ISS SISTEMA'] - df['PIS SISTEMA'] - df['COFINS SISTEMA'] - df['CSLL SISTEMA'] - df['IR SISTEMA']
df['DIFERENÇA'] = df['VALOR LIQUIDO SISTEMA'] - df['VALOR LIQUIDO SITE']

In [55]:
for cont9 in range(len(df)):
    if df['CLIENTE SISTEMA'].loc[cont9] == df['CLIENTE SITE'].loc[cont9]:
        df['COMPARAÇÃO NOME'].loc[cont9] =  'Ok'
    else:
        df['COMPARAÇÃO NOME'].loc[cont9] = 'nome divergente'

C:\Users\Gregory Toledo\AppData\Local\Temp\ipykernel_420\1649169214.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['COMPARAÇÃO NOME'].loc[cont9] =  'Ok'


In [56]:
df.to_excel('C:/Users/Gregory Toledo/conferencia notas fiscais/planilha final.xlsx',engine = 'openpyxl',encoding='utf-8',index=False)

In [59]:
df

,NF,DATA EMISSÃO,COMPETENCIA,ITEM SERVIÇO,VALOR BRUTO SISTEMA,ISS SISTEMA,PIS SISTEMA,COFINS SISTEMA,CSLL SISTEMA,IR SISTEMA,...,ISS SITE,PIS SITE,COFINS SITE,CSLL SITE,IR SITE,VALOR LIQUIDO SITE,DIFERENÇA,CLIENTE SISTEMA,CLIENTE SITE,COMPARAÇÃO NOME
0,3114,17/01/2022,2022-01-17,0,25685.66,0.0,166.96,770.56,256.86,385.28,...,0.0,166.96,770.56,256.86,385.28,24106.0,0.0,JOÃO CAMINHÕES,JOÃO CAMINHÕES,Ok
1,3115,17/01/2022,2022-01-17,0,859.24,0.0,5.59,25.77,8.59,12.89,...,0.0,5.59,25.77,8.59,12.89,806.4,0.0,JOÃO CAMINHÕES,JOÃO CAMINHÕES,Ok
2,3116,17/01/2022,2022-01-17,0,16704.66,0.0,108.58,501.14,167.05,250.57,...,0.0,108.58,501.14,167.05,250.57,15677.32,0.0,JOÃO CAMINHÕES,JOÃO CAMINHÕES,Ok
3,3117,17/01/2022,2022-01-17,0,9830.92,0.0,63.9,294.93,98.31,147.46,...,0.0,63.9,294.93,98.31,147.46,9226.32,0.0,JOÃO CAMINHÕES,JOÃO CAMINHÕES,Ok
4,3118,17/01/2022,2022-01-17,0,6345.38,0.0,41.24,190.37,63.45,95.18,...,0.0,41.24,190.37,63.45,95.18,5955.14,0.0,JOÃO CAMINHÕES,JOÃO CAMINHÕES,Ok
5,3119,17/01/2022,2022-01-17,0,34127.8,0.0,221.83,1023.83,341.28,511.92,...,0.0,221.83,1023.83,341.28,511.92,32028.94,0.0,JOÃO CAMINHÕES,JOÃO CAMINHÕES,Ok
6,3120,17/01/2022,2022-01-17,0,179.62,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,179.62,0.0,JOÃO CAMINHÕES,JOÃO CAMINHÕES,Ok
7,3121,17/01/2022,2022-01-17,0,359.24,0.0,2.34,10.77,3.59,0.0,...,0.0,2.34,10.77,3.59,0.0,342.54,0.0,JOÃO CAMINHÕES,JOÃO CAMINHÕES,Ok
8,3122,17/01/2022,2022-01-17,0,27841.1,0.0,180.97,835.23,278.41,417.62,...,0.0,180.97,835.23,278.41,417.62,26128.87,0.0,JOÃO CAMINHÕES,JOÃO CAMINHÕES,Ok
9,3123,17/01/2022,2022-01-17,0,538.86,0.0,3.5,16.17,5.39,0.0,...,0.0,3.5,16.17,5.39,0.0,513.8,0.0,JOÃO CAMINHÕES,JOÃO CAMINHÕES,Ok


,UNIDADE FATURAMENTO,MES,ANO,NR.NOTA DE SERVIÇO,DT. EMISSAO,CLIENTE,VLR.RECEITA,VLR.ISS,VLR.IRRF,VLR.COFINS,VLR.CSLL,VLR.PIS,VLR LIQUIDO
0,CONSULTORIA DE LOGISTICAS LTDA,1,2022,3121,17/01/2022,JOÃO CAMINHÕES,359.24,NaN,0.00,10.77,3.59,2.34,342.54
1,CONSULTORIA DE LOGISTICAS LTDA,1,2022,3136,18/01/2022,JOÃO CAMINHÕES,359.24,NaN,0.00,10.77,3.59,2.34,342.54
2,CONSULTORIA DE LOGISTICAS LTDA,1,2022,3141,18/01/2022,JOÃO CAMINHÕES,14663.88,NaN,219.96,439.91,146.64,95.32,13762.05
3,CONSULTORIA DE LOGISTICAS LTDA,1,2022,3145,18/01/2022,JOÃO CAMINHÕES,1281.55,NaN,19.22,38.44,12.82,8.33,1202.74
4,CONSULTORIA DE LOGISTICAS LTDA,1,2022,3114,17/01/2022,JOÃO CAMINHÕES,25685.66,NaN,385.28,770.56,256.86,166.96,24106.00
5,CONSULTORIA DE LOGISTICAS LTDA,1,2022,3118,17/01/2022,JOÃO CAMINHÕES,6345.38,NaN,95.18,190.37,63.45,41.24,5955.14
6,CONSULTORIA DE LOGISTICAS LTDA,1,2022,3142,18/01/2022,JOÃO CAMINHÕES,4402.20,NaN,66.03,132.07,44.02,28.61,4131.47
7,CONSULTORIA DE LOGISTICAS LTDA,1,2022,3124,17/01/2022,JOÃO CAMINHÕES,538.86,NaN,0.00,16.17,5.39,3.50,513.80
8,CONSULTORIA DE LOGISTICAS LTDA,1,2022,3128,17/01/2022,JOÃO CAMINHÕES,359.24,NaN,0.00,10.77,3.59,2.34,342.54
9,CONSULTORIA DE LOGISTICAS LTDA,1,2022,3129,18/01/2022,JOÃO CAMINHÕES,7184.80,NaN,107.77,215.54,71.85,46.70,6742.94
